# import

In [4]:
import collections as cl
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import measure
from skimage.segmentation import clear_border
from skimage.filters import threshold_otsu
from shapely.geometry import Polygon, MultiPolygon
from PIL import Image
import json
import cv2
import glob
import sys
import os
# print("finish import")

ModuleNotFoundError: No module named 'skimage'

In [ ]:
def info():
    tmp = cl.OrderedDict()
    tmp["description"] = "detect crater image num = 10000 For valid"
    tmp["unity_info"] = "Random seed = 656441"
    tmp["url"] = "https://www.brain.kyutech.ac.jp/~tamukoh/"
    tmp["version"] = "3.0"
    tmp["year"] = 2021
    tmp["contributor"] = "yuga yano"
    tmp["data_created"] = "2021/10/13"
    tmp["image_size"] = "550 x 550"
    return tmp

def licenses():
    tmp = cl.OrderedDict()
    tmp["id"] = 1
    tmp["url"] = "https://www.brain.kyutech.ac.jp/~tamukoh/"
    tmp["name"] = "Kyutech.Tamlab.yano"
    return tmp

def categories():
    tmps = []
    sup = ["crater"]
    cat = ["crater"]
    for i in range(len(sup)):
        tmp = cl.OrderedDict()
        tmp["id"] = i+1
        tmp["name"] = cat[i]
        tmp["supercategory"] = sup[i]
        tmps.append(tmp)
    return tmps

In [ ]:
def images(rgb_path, ignore, height, width):
    tmps = []
    files = glob.glob(rgb_path + "/*.png")
    files.sort()
    
    image_id = 0
    for i, file in enumerate(files):
        
        if i in ignore:
            pass
        else:
            tmp = cl.OrderedDict()
            tmp["license"] = 1
            tmp["id"] = image_id
            tmp["file_name"] = os.path.basename(file)
            tmp["width"] = width
            tmp["height"] = height
            tmp["date_captured"] = ""
            tmp["coco_url"] = "dummy_words"
            tmp["flickr_url"] = "dummy_words"
            tmps.append(tmp)
            image_id += 1
    return tmps

In [ ]:
def create_sub_mask_annotation_con(contour, image_id, category_id, annotation_id, is_crowd, poly, area):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
#     contours = measure.find_contours(sub_mask, 0.5)

    segmentations = []
    polygons = []
#     for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
#     for i in range(len(contour)):
#         row, col = contour[i]
#         contour[i] = (col, row)

    # Make a polygon and simplify it
#     poly = Polygon(contour)
#     poly = poly.simplify(1.0, preserve_topology=False)
    polygons.append(poly)
    segmentation = np.array(poly.exterior.coords).ravel().tolist()
    segmentations.append(segmentation)

    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    x, y, max_x, max_y = multi_poly.bounds
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
#     area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }

    return annotation

In [ ]:
def annotations(mask_path, thresh_areasize):
    print("===== make annotation start =====")
    mask_images = glob.glob(mask_path + "/*.png")
    mask_images.sort()

    # Define which colors match which categories in the images
    houseplant_id, book_id, bottle_id, lamp_id = [1, 2, 3, 4]
    crater_id = 1
    category_ids = {
        1: {
            '(77, 25, 255)': crater_id,
            '(0, 0, 255)': book_id,
        },
        2: {
            '(255, 255, 0)': bottle_id,
            '(255, 0, 128)': book_id,
            '(255, 100, 0)': lamp_id,
        }
    }

    is_crowd = 0

    # These ids will be automatically increased as we go
    annotation_id = 0
    image_id = 0
    
    rgb_image_id = 0
    # クレータが存在しない画像をデータセットから除外するため，除外する画像のidをリストで保存する
    ignore = []
    
    # Create the annotations
    annotations = []
    for mask_image_path in mask_images:
        mask_image_color = Image.open(mask_image_path).convert('RGB')
        mask_image_gray = cv2.imread(mask_image_path, 0)
        base_mask_image_name = os.path.basename(mask_image_path)
        contours = measure.find_contours(mask_image_gray, 0.5)

        # クレーターが存在しない画像が読み込まれた場合
        if len(contours) == 0:
            print("[warning_1]", base_mask_image_name, "no annotation")
            ignore.append(rgb_image_id)
            rgb_image_id += 1
            continue
    
        # 領域ごとに，ポリゴンとBboxを算出する
        for contour in contours:
            for i in range(len(contour)):
                row, col = contour[i]
                # x, yの入れ替えと小数点以下を丸める処理
                contour[i] = (round(col), round(row))
        
            polygons = []
            category_id = 1 # 今回はクラスがクレータしかないので，1で固定
            
            # ポリゴンによってエラーが出ることがあるので，try exceptで対応（エラーの原因不明）
            try:
                poly = Polygon(contour)
                poly = poly.simplify(1.0, preserve_topology=False)
#                 print(poly)
                
                # polygonが計算できた時のみ"annotations"に追加する
                if poly.is_empty == False:
                    polygons.append(poly)
                    multi_poly = MultiPolygon(polygons)
                    area = multi_poly.area
#                     print(area)
                    if area < thresh_areasize: # 小さすぎるクレーターのアノテーションデータは無視する
#                         print("[warning_3]", base_mask_image_name, "area size is too small. skipping!")
                        pass
                    else:
                        annotation = create_sub_mask_annotation_con(contour, image_id, category_id, annotation_id, is_crowd, poly, area)
                        annotations.append(annotation)
                        annotation_id += 1
            except:
                print("[warning_2]", base_mask_image_name, "skipping the counter")
        
        image_id += 1
        rgb_image_id += 1
        if image_id % 100 == 0:
            print("progress: ", image_id)
            
    print(len(mask_images),"個のファイルのうち" ,len(ignore), "個はクレータが存在しないため無視しました")
    print("最終的なデータ数は", len(mask_images)-len(ignore), "です")
    print("===== create annotation complete!! =====")
#     print(ignore)
    return annotations, ignore

# 実行関数

In [ ]:
if __name__ == "__main__":
    
    # フォルダパス
    base_path = "/home/yuga/.config/unity3d/DefaultCompany/PerceptionURP/valid/"
#     base_path = "/home/yuga/.config/unity3d/DefaultCompany/PerceptionURP/tests/ZED_720p/"
    
    mask_path = base_path + "semas"
    rgb_path = base_path + "images"
    
    # データの画像サイズ
    height = 550
    width = 550
    
    # 無視するアノテーションのエリアサイズ
    # 小さすぎるクレータのアノテーションデータはノイズになっているが可能性あるため除外する
    thresh_areasize = 1
    
    query_list = ["info", "licenses", "images", "annotations", "categories", "segment_info"]
    js = cl.OrderedDict()
    for i in range(len(query_list)):
        tmp = ""
        # Info
        if query_list[i] == "info":
            tmp = info()

        # licenses
        elif query_list[i] == "licenses":
            tmp = licenses()

#         elif query_list[i] == "images":
#             tmp = images(rgb_path)

# fix me
        elif query_list[i] == "annotations":
            tmp, ignore = annotations(mask_path, thresh_areasize)
            tmp_img = ""
            tmp_img = images(rgb_path, ignore, height, width)
            js[query_list[i-1]] = tmp_img

        elif query_list[i] == "categories":
            tmp = categories()

        # save it
        js[query_list[i]] = tmp

    print("finish")
#     print(js["images"])

# json書き込み
- 一回だけだと書き込めないバグがあるので，二回実行する

In [ ]:
# write
# 書き出すjsonのファイル名
json_name = base_path + "valid_moon.json"

fw = open(json_name, 'w')
json.dump(js, fw)
# json.dump(js, fw, indent=2)

# テスト用
- 書き込みが成功していれば，categoriesを確認することができる
- errorが出れば，書き込みに失敗しているので，書き込みをもう一回実行する

In [ ]:
with open(json_name) as f:
    df = json.load(f)
# print(len(df["images"]))
print("last_image_id =", df["images"][-1]["id"])
print("info = ", df["info"])

In [ ]:
# with open("train_moon.json") as f:
#     df = json.load(f)
print(df["annotations"][13]["segmentation"])

# 必要なし

In [ ]:
tmp = ""
tmp = info()
js[0] = tmp
print(js[0])

In [ ]:
# plant_book_mask_image = Image.open('/home/yuga/.config/unity3d/DefaultCompany/PerceptionURP/train/semas/segmentation_2.png').convert('RGB')
# bottle_book_mask_image = Image.open('/home/yuga/.config/unity3d/DefaultCompany/PerceptionURP/train/semas/segmentation_3.png').convert('RGB')

plant_book_mask_image = Image.open('images/plant_book_mask.png')
bottle_book_mask_image = Image.open('images/bottle_book_mask.png')

mask_images = [plant_book_mask_image, bottle_book_mask_image]

# Define which colors match which categories in the images
houseplant_id, book_id, bottle_id, lamp_id = [1, 2, 3, 4]
category_ids = {
    1: {
        '(0, 255, 0)': houseplant_id,
        '(0, 0, 255)': book_id,
    },
    2: {
        '(255, 255, 0)': bottle_id,
        '(255, 0, 128)': book_id,
        '(255, 100, 0)': lamp_id,
    }
}

is_crowd = 0

# These ids will be automatically increased as we go
annotation_id = 1
image_id = 1

# Create the annotations
annotations = []
for mask_image in mask_images:
    print(mask_image)
    sub_masks = create_sub_masks(mask_image)
    for color, sub_mask in sub_masks.items():
        category_id = category_ids[image_id][color]
        print(sub_mask)
        plt.imshow(sub_mask)
        plt.show(sub_mask)
        annotation = create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd)
        annotations.append(annotation)
        annotation_id += 1
    image_id += 1

print(json.dumps(annotations))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import ndimage
from skimage import measure
from skimage.segmentation import clear_border
from skimage.filters import threshold_otsu
plt.rcParams['font.size']=12
plt.rcParams['font.family'] = 'sans-serif'

#make data
n = 6
s = 256
im = np.zeros((s, s))
points = (s*np.random.random((2, n**2))).astype(int)
im[points[0], points[1]] = 1
im = ndimage.gaussian_filter(im, sigma=s/(5.*n))

thresh = threshold_otsu(im)
im = im > thresh 
im = clear_border(im)

print(im)
contours = measure.find_contours(im, 0.5)

fig, ax = plt.subplots(figsize=(6,6))
for n, contour in enumerate(contours):
    ax.plot(contour[:, 1], contour[:, 0], linewidth=4)
    ax.text(contour[0, 1], contour[0, 0],n)
    
ax.imshow(im, cmap=plt.cm.cividis, interpolation='gaussian',origin='lower',alpha=0.5)
plt.savefig('findcontours05.png',dpi=100)
plt.show() 

In [ ]:
def create_sub_mask_annotation(sub_mask, image_id, category_id, annotation_id, is_crowd):
    # Find contours (boundary lines) around each sub-mask
    # Note: there could be multiple contours if the object
    # is partially occluded. (E.g. an elephant behind a tree)
    contours = measure.find_contours(sub_mask, 0.5)

    segmentations = []
    polygons = []
    for contour in contours:
        # Flip from (row, col) representation to (x, y)
        # and subtract the padding pixel
        for i in range(len(contour)):
            row, col = contour[i]
            contour[i] = (col - 1, row - 1)

        # Make a polygon and simplify it
        poly = Polygon(contour)
        poly = poly.simplify(1.0, preserve_topology=False)
        polygons.append(poly)
        segmentation = np.array(poly.exterior.coords).ravel().tolist()
        segmentations.append(segmentation)

    # Combine the polygons to calculate the bounding box and area
    multi_poly = MultiPolygon(polygons)
    x, y, max_x, max_y = multi_poly.bounds
    width = max_x - x
    height = max_y - y
    bbox = (x, y, width, height)
    area = multi_poly.area

    annotation = {
        'segmentation': segmentations,
        'iscrowd': is_crowd,
        'image_id': image_id,
        'category_id': category_id,
        'id': annotation_id,
        'bbox': bbox,
        'area': area
    }

    return annotation

In [ ]:
def create_sub_masks(mask_image):
    width, height = mask_image.size

    # Initialize a dictionary of sub-masks indexed by RGB colors
    sub_masks = {}
    for x in range(width):
        for y in range(height):
            # Get the RGB values of the pixel
            pixel = mask_image.getpixel((x,y))[:3]

            # If the pixel is not black...
            if pixel != (0, 0, 0):
                # Check to see if we've created a sub-mask...
                pixel_str = str(pixel)
                sub_mask = sub_masks.get(pixel_str)
                if sub_mask is None:
                   # Create a sub-mask (one bit per pixel) and add to the dictionary
                    # Note: we add 1 pixel of padding in each direction
                    # because the contours module doesn't handle cases
                    # where pixels bleed to the edge of the image
                    sub_masks[pixel_str] = Image.new('1', (width+2, height+2))

                # Set the pixel value to 1 (default is 0), accounting for padding
                sub_masks[pixel_str].putpixel((x+1, y+1), 1)

    return sub_masks